## 第三節　ルームエアコンディショナー

1．適用範囲～4．記号及び単位は省略

### 5. 最大暖房出力

$$\large
Q_{max,H,d,t} = Qr_{max,H,d,t} \times q_{rtd,H} \times C_{af,H} \times C_{df,H,d,t} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$Qr_{max,H,d,t}$：日付$d$の時刻$t$における最大暖房出力比  
$q_{rtd,H}$：定格暖房能力(W)  
$C_{af,H}$：室内機吹き出し風量に関する暖房出力補正係数  
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数  

$$\large
Qr_{max,H,d,t} = a_2 \times (\theta_{ex,d,t} - 7)^2 + a_1 \times (\theta_{ex,d,t} - 7) + a_0
$$

$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\Biggl(\begin{matrix}
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) \times qr_{max,H} + 
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    0.000181\\ 
    0.002322\\ 
    0.003556\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    -0.000184\\ 
    0.013904\\ 
    0.993431\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    -0.000173\\ 
    -0.003980\\ 
    -0.002870\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    0.000367\\ 
    0.003983\\ 
    0.006376\\ 
    \end{matrix}
\Biggr)
$$

$qr_{max,H}$：最大暖房能力比  
$q_{rtd,C}$：定格冷房能力(W)  

$$\large
qr_{max,H} = \frac{q_{max,H}}{q_{rtd,H}}
$$

$q_{max,H}$：最大暖房能力(W)  
$q_{rtd,H}$：定格暖房能力(W)  

室内機吹き出し風量に関する暖房時の能力補正係数$C_{af,H}$は0.8とする。  
デフロストに関する暖房出力補正係数$C_{df,H}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、 それ以外の条件においては1.0とする。

### 6. 暖房エネルギー消費量

#### 6.1 消費電力量

日付$d$の時刻$t$における補正処理暖房負荷$Q'_{T,H,d,t}$が0の場合、$E_{E,H,d,t}$ = 0とする。

$$\large
E_{E,H,d,t} = f_{H,\theta} \Biggl( \frac{Q'_{T,H,d,t}}{q_{max,H} \times 3600 \times 10^{-6}} \Biggr) \div 
f_{H,7} \Biggl( \frac{1}{qr_{max,H}} \Biggr) \times \frac{q_{rtd,H}}{e_{rtd,H}} \times 10^{-3}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$f_{H,\theta}$：基準入出力関数  
$Q'_{T,H,d,t}$：日付$d$の時刻$t$における補正処理暖房負荷(MJ/h)    
$q_{max,H}$：最大暖房能力(W)  
$qr_{max,H}$：最大暖房能力比  
$q_{rtd,H}$：定格暖房能力(W)  
$e_{rtd,H}$：定格暖房エネルギー消費効率  

$$\large
f_{H,\theta}(x) = a_{4}x^{4} + a_{3}x^{3} + a_{2}x^{2} + a_{1}x + a_{0}
$$

\begin{equation*}
\left(\begin{matrix}
    a_4\\
    a_3\\
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\right) = 
\left(\begin{matrix}
    p_{42} & p_{41} & p_{40}\\
    p_{32} & p_{31} & p_{30}\\
    p_{22} & p_{21} & p_{20}\\
    p_{12} & p_{11} & p_{10}\\
    p_{02} & p_{01} & p_{00}\\
    \end{matrix}
\right)
\left(\begin{matrix}
    \theta_{ex}^2\\ 
    \theta_{ex}\\ 
    1\\ 
    \end{matrix}
\right)
\end{equation*}

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

\begin{equation*}
\large
p_{i} = s_{i} \times q_{rtd,C} \times 10^{-3} + t_{i}
\end{equation*}

$q_{rtd,C}$：定格冷房能力(W)  

<div style="text-align: center;font-weight: bold;">表　係数 $s_{i}$ 及び $t_{i}$</div>

|係数|値|係数|値|係数|値|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|-|-|-|-|-|-|
|$S_{42}$|-0.00236|$t_{42}$|0.01324|$S_{41}$|0.08418|$t_{41}$|-0.47143|$S_{40}$|-1.16944|$t_{40}$|6.54886|
|$S_{32}$|0.00427|$t_{32}$|-0.02392|$S_{31}$|-0.19226|$t_{31}$|0.94213|$S_{30}$|2.58632|$t_{30}$|-12.85618|
|$S_{22}$|-0.00275|$t_{22}$|0.01542|$S_{21}$|0.14947|$t_{21}$|-0.68303|$S_{20}$|-2.03594|$t_{20}$|10.60561|
|$S_{12}$|0.00063|$t_{12}$|-0.00351|$S_{11}$|-0.02865|$t_{11}$|0.10522|$S_{10}$|0.37336|$t_{10}$|-1.09499|
|$S_{02}$|-0.00005|$t_{02}$|0.00028|$S_{01}$|0.00184|$t_{01}$|-0.01090|$S_{00}$|-0.09609|$t_{00}$|0.59229|


$q_{rtd,C} \leq 2200$ の場合

$$\large
p_{i} = p_{i,A}
$$

$2200 < q_{rtd,C} \leq 4000$ の場合

$$\large
p_{i} = p_{i,A} \times \frac{4000-q_{rtd,C}}{4000-2200} + p_{i,B} \times \frac{q_{rtd,C}-2200}{4000-2200}
$$

$4000 < q_{rtd,C} < 7100$ の場合

$$\large
p_{i} = p_{i,B} \times \frac{7100-q_{rtd,C}}{7100-4000} + p_{i,C} \times \frac{q_{rtd,C}-4000}{7100-4000}
$$

$7100 \leq q_{rtd,C}$ の場合

$$\large
p_{i} = p_{i,C}
$$

$p_{i,A}$：表により表される係数  
$p_{i,B}$：表により表される係数  
$p_{i,C}$：表により表される係数  

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,A}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.000056|$p_{41}$|0.000786|$p_{40}$|0.071625|
|$p_{32}$|-0.000145|$p_{31}$|0.003337|$p_{30}$|-0.143643|
|$p_{22}$|-0.000240|$p_{21}$|-0.029471|$p_{20}$|1.954343|
|$p_{12}$|-0.000035|$p_{11}$|-0.050909|$p_{10}$|1.389751|
|$p_{02}$|0.0|$p_{01}$|0.0|$p_{00}$|0.076800|  


<div style="text-align: center;font-weight: bold;">表　係数$p_{i,B}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|0.000108|$p_{41}$|-0.035658|$p_{40}$|3.063873|
|$p_{32}$|-0.000017|$p_{31}$|0.062546|$p_{30}$|-5.471556|
|$p_{22}$|-0.000245|$p_{21}$|-0.025126|$p_{20}$|4.057590|
|$p_{12}$|0.000323|$p_{11}$|-0.021166|$p_{10}$|0.575459|
|$p_{02}$|0.0|$p_{01}$|0.000330|$p_{00}$|0.047500| 


<div style="text-align: center;font-weight: bold;">表　係数$p_{i,C}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.001465|$p_{41}$|-0.030500|$p_{40}$|1.920431|
|$p_{32}$|0.002824|$p_{31}$|0.041081|$p_{30}$|-1.835302|
|$p_{22}$|-0.001929|$p_{21}$|-0.009738|$p_{20}$|1.582898|
|$p_{12}$|0.000616|$p_{11}$|-0.014239|$p_{10}$|0.546204|
|$p_{02}$|0.0|$p_{01}$|-0.000110|$p_{00}$|0.023100|


$$\large
Q'_{T,H,d,t} = Q_{T,H,d,t} \times \frac{1}{C_{af,H} \times C_{df}}
$$

$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の処理暖房負荷(MJ/h)

#### 6.2 ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

#### 6.3 灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

#### 6.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

### 7. 最大冷房出力

#### 7.1 最大冷房出力の計算

$$\large
Q_{max,C,d,t} = Qr_{max,C,d,t} \times q_{rtd,C} \times C_{af,C} \times C_{hm,C} \times 3600 \times 10^{-6}
$$

$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$Qr_{max,C,d,t}$：日付$d$の時刻$t$における最大冷房出力比  
$q_{rtd,C}$：定格冷房能力(W)  
$C_{af,C}$：室内機吹き出し風量に関する冷房出力補正係数  
$C_{hm,C}$：室内機吸い込み湿度に関する冷房出力補正係数  

$$\large
Qr_{max,C,d,t} = a_2 \times (\theta_{ex,d,t} - 35)^2 + a_1 \times (\theta_{ex,d,t} - 35) + a_0
$$

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\Biggl(\begin{matrix}
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) \times qr_{max,C} + 
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    0.000182\\ 
    0.003527\\ 
    -0.011490\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    -0.001480\\ 
    -0.023000\\ 
    1.024328\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    -0.000350\\ 
    -0.001280\\ 
    0.004772\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    0.000800\\ 
    0.003621\\ 
    -0.011170\\ 
    \end{matrix}
\Biggr)
$$

$qr_{max,C}$：最大冷房能力比  
$q_{rtd,C}$：定格冷房能力(W)  

$$\large
qr_{max,C} = \frac{q_{max,C}}{q_{rtd,C}}
$$

$q_{max,C}$：最大冷房能力(W)  
$q_{rtd,C}$：定格冷房能力(W)  

室内機吹き出し風量に関する冷房時の能力補正係数$C_{af,C}$は0.85とする。  
室内機吸い込み湿度に関する冷房能力補正係数$C_{hm,C}$は1.15とする。

#### 7.2 最大冷房顕熱出力及び最大冷房潜熱出力の計算

$$\large
Q_{max,CS,d,t} = Q_{max,C,d,t} \times SHF'_{d,t}
$$

$$\large
Q_{max,CL,d,t} = min(Q_{max,C,d,t} \times (1 - SHF'_{d,t}),  L'_{CL,d,t})
$$

$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房顕熱出力(MJ/h)  
$Q_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱出力(MJ/h)  
$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$SHF'_{d,t}$：日付$d$の時刻$t$における冷房負荷補正顕熱比  

$$\large
SHF'_{d,t} = \left\{ \begin{array}{ll}
    \frac{L_{CS,d,t}}{L'_{C,d,t}} & (L'_{C,d,t} \neq 0)\\ 
    0 & (L'_{C,d,t} = 0)\\ 
    \end{array}
\right.
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房負荷(MJ/h)  

$$\large
L'_{C,d,t} = L_{CS,d,t} + L'_{CL,d,t}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房潜熱負荷(MJ/h)  

$$\large
L'_{CL,d,t} = min(L_{max,CL,d,t},  L_{CL,d,t})
$$

$L_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱負荷(MJ/h)  
$L_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷(MJ/h)  

$$\large
L_{max,CL,d,t} = L_{CS,d,t} \times \frac{1 - SHF_{L,min,c}}{SHF_{L,min,c}}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$SHF_{L,min,c}$：冷房負荷最小顕熱比  

冷房負荷最小顕熱比$SHF_{L,min,c}$は0.4とする。

### 8. 冷房エネルギー消費量

#### 8.1 消費電力量

日付$d$の時刻$t$における補正処理冷房負荷 $Q'_{T,c,d,t}$が0の場合、$E_{E,C,d,t}$ = 0とする。

$$\large
E_{E,C,d,t} = f_{C,\theta} \Biggl( \frac{Q'_{T,c,d,t}}{q_{max,C} \times 3600 \times 10^{-6}} \Biggr) \div 
f_{C,35} \Biggl( \frac{1}{qr_{max,C}} \Biggr) \times \frac{q_{rtd,C}}{e_{rtd,C}} \times 10^{-3}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$f_{C,\theta}$：基準入出力関数    
$Q'_{T,c,d,t}$：日付$d$の時刻$t$における補正処理冷房負荷(MJ/h)    
$q_{max,C}$：最大冷房能力(W)    
$q_{rtd,C}$：定格冷房能力(W)    
$e_{rtd,C}$：定格冷房エネルギー消費効率  

$$\large
f_{C,\theta}(x) = a_{4}x^{4} + a_{3}x^{3} + a_{2}x^{2} + a_{1}x + a_{0}
$$

\begin{equation*}
\left(\begin{matrix}
    a_4\\
    a_3\\
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\right) = 
\left(\begin{matrix}
    p_{42} & p_{41} & p_{40}\\
    p_{32} & p_{31} & p_{30}\\
    p_{22} & p_{21} & p_{20}\\
    p_{12} & p_{11} & p_{10}\\
    p_{02} & p_{01} & p_{00}\\
    \end{matrix}
\right)
\left(\begin{matrix}
    \theta_{ex}^2\\ 
    \theta_{ex}\\ 
    1\\ 
    \end{matrix}
\right)
\end{equation*}

容量可変型コンプレッサー搭載ルームエアコンディショナーでないルームエアコンディショナーについては、係数$p_{i}$は式により与えられ、係数$s_{i}$及び$t_{i}$は表より表される。
$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\begin{equation*}
\large
p_{i} = s_{i} \times q_{rtd,C} \times 10^{-3} + t_{i}
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

<div style="text-align: center;font-weight: bold;">表　係数$s_{i}$及び$t_{i}$</div>

|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|
|-|-|-|-|-|-|-|-|-|-|-|-|
|$S_{42}$|0.00000|$t_{42}$|0.00000|$S_{41}$|0.00000|$t_{41}$|0.00000|$S_{40}$|0.00000|$t_{40}$|0.00000|
|$S_{32}$|0.00000|$t_{32}$|0.00000|$S_{31}$|-0.00036|$t_{31}$|0.05080|$S_{30}$|-0.20346|$t_{30}$|0.47765|
|$S_{22}$|0.00000|$t_{22}$|0.00000|$S_{21}$|0.00227|$t_{21}$|-0.03952|$S_{20}$|0.04115|$t_{20}$|0.23099|
|$S_{12}$|0.00000|$t_{12}$|0.00000|$S_{11}$|-0.00911|$t_{11}$|0.07102|$S_{10}$|0.14950|$t_{10}$|-1.07335|
|$S_{02}$|0.00000|$t_{02}$|0.00000|$S_{01}$|0.00044|$t_{01}$|-0.00214|$S_{00}$|-0.06250|$t_{00}$|0.35150|


容量可変型コンプレッサー搭載ルームエアコンディショナーについては式により表される。

$q_{rtd,C} \leq 2200$ の場合

$$\large
p_{i} = p_{i,A}
$$

$2200 < q_{rtd,C} \leq 4000$ の場合

$$\large
p_{i} = p_{i,A} \times \frac{4000-q_{rtd,C}}{4000-2200} + p_{i,B} \times \frac{q_{rtd,C}-2200}{4000-2200}
$$

$4000 < q_{rtd,C} < 7100$ の場合

$$\large
p_{i} = p_{i,B} \times \frac{7100-q_{rtd,C}}{7100-4000} + p_{i,C} \times \frac{q_{rtd,C}-4000}{7100-4000}
$$

$7100 \leq q_{rtd,C}$ の場合

$$\large
p_{i} = p_{i,C}
$$

$p_{i,A}$：表により表される係数  
$p_{i,B}$：表により表される係数  
$p_{i,C}$：表により表される係数  

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,A}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.0004078|$p_{41}$|0.01035|$p_{40}$|-0.03248|
|$p_{32}$|0.0|$p_{31}$|0.04099|$p_{30}$|-0.81889|
|$p_{22}$|0.0|$p_{21}$|-0.04615|$p_{20}$|2.10666|
|$p_{12}$|0.0013382|$p_{11}$|-0.01179|$p_{10}$|-0.41778|
|$p_{02}$|0.0000000|$p_{01}$|-0.00102|$p_{00}$|0.09270|


<div style="text-align: center;font-weight: bold;">表　係数$p_{i,B}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.000056|$p_{41}$|-0.003539|$p_{40}$|-0.430566|
|$p_{32}$|0.0|$p_{31}$|0.015237|$p_{30}$|1.188850|
|$p_{22}$|0.0|$p_{21}$|0.000527|$p_{20}$|-0.304645|
|$p_{12}$|-0.000179|$p_{11}$|0.020543|$p_{10}$|0.130373|
|$p_{02}$|0.0|$p_{01}$|0.000240|$p_{00}$|0.013500|


<div style="text-align: center;font-weight: bold;">表　係数$p_{i,C}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.0001598|$p_{41}$|0.004848|$p_{40}$|0.047097|
|$p_{32}$|0.0|$p_{31}$|0.016675|$p_{30}$|0.362141|
|$p_{22}$|0.0|$p_{21}$|-0.008134|$p_{20}$|-0.023535|
|$p_{12}$|-0.0000772|$p_{11}$|0.012558|$p_{10}$|0.056185|
|$p_{02}$|0.0|$p_{01}$|-0.000110|$p_{00}$|0.010300|


$$\large
Q'_{T,C,d,t} = (Q_{T,CS,d,t} + Q_{T,CL,d,t}) \times \frac{1}{C_{hm,C} \times C_{af,C}}
$$

$Q_{T,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の処理冷房潜熱負荷(MJ/h)  
$C_{hm,C}$：室内機吸い込み湿度に関する冷房出力補正係数  
$C_{af,C}$：室内機吹き出し風量に関する冷房出力補正係数  

#### 8.2 ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,C,d,t}$は0とする。

#### 8.3 灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,C,d,t}$は0とする。

#### 8.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,C,d,t}$は0とする。

### 付録 A 機器の性能を表す仕様の決定方法

#### A.1 記号と単位

省略

#### A.2 定格能力

$$
\begin{equation*}
\large
q_{rtd,H} = 1.2090 \times q_{rtd,C} - 85.1
\end{equation*}
$$

$q_{rtd,H}$：定格暖房能力(W)    
$q_{rtd,C}$：定格冷房能力(W)    

定格冷房能力$q_{rtd,C}$が5600(W)を超える場合については、定格冷房能力$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\begin{equation*}
\large
q_{rtd,C} = 190.5 \times A_{HCZ} + 45.6
\end{equation*}
$$

$A_{HCZ}$：ルームエアコンディショナーにより暖冷房される暖冷房区画の床面積(m<sup>2</sup>)

#### A.3 最大能力

$$
\begin{equation*}
\large
q_{max,H} = 1.7597 \times q_{max,C} - 413.7
\end{equation*}
$$

$q_{max,H}$：最大暖房能力(W)    
$q_{max,C}$：最大冷房能力(W)    

定格暖房能力$q_{rtd,H}$が最大暖房能力$q_{max,H}$を超える場合については、最大暖房能力$q_{max,H}$は定格暖房能力$q_{rtd,H}$に等しいとする。

$$
\begin{equation*}
\large
q_{max,C} = 0.8462 \times q_{rtd,C} + 1205.9
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

定格冷房能力$q_{rtd,C}$が最大冷房能力$q_{max,C}$を超える場合については、最大冷房能力$q_{max,C}$は定格冷房能力$q_{rtd,C}$に等しいとする。

#### A.4 定格エネルギー消費効率

$$
\begin{equation*}
\large
e_{rtd,H} = 0.77 \times e_{rtd,C} + 1.66
\end{equation*}
$$

$e_{rtd,H}$：定格暖房エネルギー消費効率    
$e_{rtd,C}$：定格冷房エネルギー消費効率    

区分（い）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.553 \times 10^{−3} \times q_{rtd,C} + 6.34
\end{equation*}
$$

区分（ろ）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.504 \times 10^{−3} \times q_{rtd,C} + 5.88
\end{equation*}
$$

区分（は）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.473 \times 10^{−3} \times q_{rtd,C} + 5.50
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

#### A.5 容量可変型コンプレッサー搭載の有無

省略

#### A.6 複数のルームエアコンディショナーが設置される場合の仕様の決定方法

省略

### 付録 B 容量可変型コンプレッサー搭載ルームエアコンディショナーの定義と判別方法

省略

### 付録 C （参考）可変気筒方式コンプレッサー搭載ルームエアコンディショナーの判別方法

省略